# Extract data from caNano exported json file
Rui He, 5/19/2022

for my dear Weina Miao Miao

In [1]:
# !pip install pandas==1.4.2

In [2]:
import json
import os
import csv
import pandas as pd
import numpy as np
import re

In [3]:
data_path=os.path.abspath("imaging_function.json")
output_path=os.path.abspath("imaging_Function_extract.csv")
data_main = pd.read_json(data_path)

data_main.head()

,sample
0,"{'GeneralInfo': {'sampleId': 109969408, 'sampl..."
1,"{'GeneralInfo': {'sampleId': 96075802, 'sample..."
2,"{'GeneralInfo': {'sampleId': 96075785, 'sample..."
3,"{'GeneralInfo': {'sampleId': 96075784, 'sample..."
4,"{'GeneralInfo': {'sampleId': 96075783, 'sample..."


In [4]:
# Inspect data
# data_main.loc[0][0]

In [5]:
# Get data size
data_length=data_main.size
print(data_length)

639


In [6]:
def check_key_exist(data_frame, key):     
    if key in list(data_frame.keys()):
        return True
    else:
        return False

In [7]:
def Check_and_extract(data_frame,key,palceholder):
    if key in list(data_frame.keys()):
        targetvar=data_frame[key]
    else:
        targetvar=palceholder
    return targetvar
        

In [8]:
headers=['samplename','nameid','function entity','property name', \
        'property type','function type','function description',\
         'full description','abstract','link']
output_table=[]

In [10]:
# # data_main.loc[0][0]['Composition']['functionalizingentity']['Fluorophore'][0]['Desc']
# data_direct=data_main.loc[39][0]['Publication']['category2Publications']
# keystr=list(data_direct.keys())[0]
# citation=data_direct[keystr][0]['displayName']
# link=re.search('href=(.*)PMID', citation)
# print(citation)
# print(link.group(1))

In [11]:
counter=1
try:
    for i in range(0,data_length,1):
        item=data_main.loc[i][0]

        sampleId=item['GeneralInfo']['sampleId']

        sampleName=item['GeneralInfo']['sampleName']

        funcEntity=item['Composition']['functionalizingentity']

        publication=item['Publication']
        if len(publication)>0:     
            if 'category2Publications' in list(publication.keys()):
                data_direct=publication['category2Publications']
                keystr_list=list(data_direct.keys())
                if len(keystr_list)>0:
                    ab_entry=data_direct[keystr_list[0]][0]
                    if 'description' in list(ab_entry.keys()):
                        abstract=ab_entry['description']
                    else:
                        abstract='NaN'

                    if 'displayName' in list(ab_entry.keys()):
                        citation=ab_entry['displayName']
                        link_find=re.search('href=(.*)>PMID', citation)
                        if link_find is None:
                            link_find=re.search('href=(.*)>DOI', citation)
                            if link_find is None:
                                link_find=re.search('href=(.*) target', citation)
                        link=link_find.group(1)
                    else:
                        link='NaN'

                else:
                    abstract='NaN'
                    link='NaN'
            else:
                abstract='NaN'
                link='NaN'

        else:
            abstract='NaN'
            link='NaN'





        # if sampleId==11337728:
        #     print(counter)
        #     break
        if len(funcEntity)!=0:
            for items in funcEntity:
                subitem=funcEntity[items][0]
                # print(subitem)
                # os.system("pause")
                prop_name=subitem['Name']               


                if check_key_exist(subitem,'Decription'):
                    if len(subitem['Decription'])==0:
                        full_des='NaN'
                    else: 
                        full_des=Check_and_extract(subitem, 'type','NaN')
                else:
                    full_des='NaN'



                if check_key_exist(subitem,'Properties'):
                    if len(subitem['Properties'])==0:
                        prop_type='NaN'
                    else: 
                        prop_type=Check_and_extract(subitem['Properties'], 'type','NaN')
                else:
                    prop_type='NaN'


                if check_key_exist(subitem,'Functions'):
                    if len(subitem['Functions'])==0:
                        func_type='NaN'
                        func_des='NaN'
                    else:
                        for fuc_item in subitem['Functions']:
                            func_type=fuc_item['Type']                        
                            func_des=Check_and_extract(fuc_item, 'FunctionDescription','NaN')   

                            to_write=[sampleName,sampleId,items,prop_name,prop_type,\
                                      func_type,func_des,full_des,abstract,link]
                            if len(output_table)>0:
                                if output_table[-1]!=to_write:
                                    output_table.append(to_write) 
                            else:
                                output_table.append(to_write)



                else:
                    func_type='NaN'
                    func_des='NaN'
                    to_write=[sampleName,sampleId,items,prop_name,prop_type,\
                              func_type,func_des,full_des,abstract,link]
                    if len(output_table)>0:
                        if output_table[-1]!=to_write:
                            output_table.append(to_write)
                    else:
                        output_table.append(to_write)


        else:
            to_write=[sampleName,sampleId,'NaN','NaN','NaN','NaN','NaN','NaN','NaN','NaN']
            if len(output_table)>0:
                if output_table[-1]!=to_write:
                    output_table.append(to_write)
            else:
                output_table.append(to_write)
        counter+=1
except Exception as e: 
    print(counter)
    print(e)
    

                


In [12]:
# output_table[10]

In [13]:
# for item in output_table:
#     tracer='UNC-HKangPR2005-05'
#     if tracer in item:
#         print(item)

In [14]:
with open(output_path, 'w',newline="") as output:
    writer = csv.DictWriter(output, fieldnames = headers)
    writer.writeheader()
    writer = csv.writer(output)
    writer.writerows(output_table)